In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [ ]:
import nltk
from nltk.corpus import wordnet

In [ ]:
PATH = ["../dataset/train.tsv","../dataset/valid.tsv","../dataset/test.tsv"]

In [ ]:
train = pd.read_csv(PATH[0], sep="\t")
valid = pd.read_csv(PATH[1], sep="\t")
test = pd.read_csv(PATH[2], sep="\t")

In [ ]:
train_data = train.preprocessed_tweet_text_no_link
train_data = train_data.append(valid.preprocessed_tweet_text_no_link)
train_data = train_data.reset_index(drop=True)
train_data.shape

In [ ]:
test_data = test.preprocessed_tweet_text_no_link
test_data = test_data.reset_index(drop=True)
test_data.shape

In [ ]:
vectorizer = TextVectorization(max_tokens=26000, output_sequence_length = 128, split='whitespace', standardize='lower_and_strip_punctuation', output_mode="int")
ds = tf.data.Dataset.from_tensor_slices(train_data)
vectorizer.adapt(ds.batch(64))

In [ ]:
vectorizer2 = TextVectorization(max_tokens=26000, output_sequence_length = 128, split='whitespace', standardize='lower_and_strip_punctuation', output_mode="int")
ds2 = tf.data.Dataset.from_tensor_slices(test_data)
vectorizer2.adapt(ds2.batch(64))

In [ ]:
vocab_train = set(vectorizer.get_vocabulary())
vocab_test = set(vectorizer2.get_vocabulary())

In [ ]:
print(len(vocab_test), len(vocab_train))

In [ ]:
len(vocab_test - vocab_train)

In [ ]:
syns = wordnet.synsets("test")
print(syns[0].definition())

In [ ]:
unk = vocab_test - vocab_train

In [ ]:
definition_data= []
for el in unk:
    syns = wordnet.synsets(el)
    if len(syns) != 0:
        sentence = "definition of " + el + " is " +syns[0].definition()
        definition_data.append(sentence)
len(definition_data)

In [ ]:
df_augm_helper = pd.DataFrame(definition_data, columns=["preprocessed_tweet_text_no_link"])

In [ ]:
df_augm_helper["q1_label"] = 1

In [ ]:
df_augm_helper.to_csv("../dataset/augm_helper.tsv", index=False, sep="\t")